# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

Из-за того что у нас нет тестовой выборки, сначала разделим данные на 60-40% и 40% поделим на пополам

In [4]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [5]:
target.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [6]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [7]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

## Исследуйте модели

Если у нас задача обучения модели на классификацию, то будем исследовать 3 модели: классификацию по линейной регрессии, классификацию по дереву решений, и классификацию по случайному лесу решений. Начнем с дерева решений

In [8]:
features_train.shape

(1928, 4)

In [9]:
features_valid_test.shape

(1286, 4)

In [10]:
features_valid.shape

(643, 4)

###  Дерево решений

Подберем гиперпараметр по максимальной глубине дерева

In [11]:
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid)) 

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819


Хоть и на обучающейся выборке чем больше глубина тем лучше результат, но мне кажется на тестовой выборке точность будет меньше, так как модель будет переобучаться. Поэтому максимальную глубину я бы выбрал 3

Попробую использовать GreedSeacrhCV для дерева решений, а для случайного леса попробую RandomizedSearchCV

In [12]:
model = DecisionTreeClassifier(random_state=12345)
    

param_dict = {'max_depth': range(1, 10)}

grid = GridSearchCV(model, param_dict, cv = 5, scoring='accuracy', n_jobs = -1)

grid.fit(features_train, target_train)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=12345),
             n_jobs=-1, param_grid={'max_depth': range(1, 10)},
             scoring='accuracy')

In [13]:
grid.best_estimator_

DecisionTreeClassifier(max_depth=3, random_state=12345)

In [14]:
grid.best_score_

0.8018706681919119

Ну и GridSearcgCV показал тот же результат

### Случайный лес

Подберем гиперпараметры по максимальной глубине леса и количеству деревьев

In [15]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid) 
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 10


In [16]:
model = RandomForestClassifier(random_state=12345)
    

param_dict = {'max_depth': range(1, 10),
             'n_estimators': range(10, 201, 10)}

grid = RandomizedSearchCV(model, param_dict, cv = 5, scoring='accuracy', n_jobs = -1)

grid.fit(features_train, target_train)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
                   n_jobs=-1,
                   param_distributions={'max_depth': range(1, 10),
                                        'n_estimators': range(10, 201, 10)},
                   scoring='accuracy')

In [17]:
grid.best_estimator_

RandomForestClassifier(max_depth=9, n_estimators=50, random_state=12345)

In [19]:
grid.best_score_

0.8195087813740664

Результат даже лучше получился, чем в прошлый раз

### Логистическая регрессия

In [20]:
model = LogisticRegression(random_state=12345)

model.fit(features_train, target_train)

result = model.score(features_valid, target_valid)

In [21]:
result

0.7107309486780715

Исходя из решений получается что больший accuracy является у случайного леса решений, при гиппер параметрах количества деревьев - 40, и максимальной глубины 10

## Проверьте модель на тестовой выборке

Проверим на тестовой выборке случайный лес

In [22]:
model_rand = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=10)
model_rand.fit(features_train, target_train) 
predictions_test_rand = model_rand.predict(features_test)

accuracy_score(target_test, predictions_test_rand) 

0.8009331259720062

Проверим на тестовой выборке дерево решений

In [23]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=3)
model_tree.fit(features_train, target_train) 
predictions_test_tree = model_tree.predict(features_test)

accuracy_score(target_test, predictions_test_tree) 

0.7791601866251944

Проверим на тестовой выборке логическую регрессию

In [24]:
model_log = LogisticRegression(random_state=12345)

model_log.fit(features_train, target_train)

model_log.score(features_test, target_test)

0.6842923794712286

И даже на тестовой выборке случайный лес работает лучше всего для данной выборке 

## (бонус) Проверьте модели на адекватность

Для проверки на адекватность буду использовать DummyClassifier

In [30]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

0.6924273858921162